In [1]:
import pandas as pd
import numpy as np

In [2]:
# hyperparamters
SHEET_PATH = 'TT Optimizaotion.xlsx'
MAX_GAP = 7
MIN_GAP = 2
MAX_TIMESLOT_DATE = 8
N_COURSES = 10

In [3]:
df_students = pd.read_excel(SHEET_PATH, sheet_name="Students")
df_timetable = pd.read_excel(SHEET_PATH, sheet_name="Courses1", index_col="#")

In [4]:
df_students

,Name,C1,C2,C3,C4,C5,C6,C7,Concat
0,Ziad Amerr,Signals,AI,Compilers,Algorithms,Testing,DB,HR,"Signals, AI, Compilers, Algorithms, Testing, D..."
1,Peter Ehab,Signals,Algorithms,DB,AI,DSA,HR,NaN,"Signals, Algorithms, DB, AI, DSA, HR,"
2,Mostafa Khalil,Testing,AI,Algorithms,HR,Compilers,Signals,NaN,"Testing, AI, Algorithms, HR, Compilers, Signals,"
3,Abdelrahman Salah,Testing,Algorithms,Signals,Compilers,DB,HR,AI,"Testing, Algorithms, Signals, Compilers, DB, H..."
4,Abdelrahman Soliman,Signals,Compilers,EDA,HR,Algorithms,DB,NaN,"Signals, Compilers, EDA, HR, Algorithms, DB,"
5,Abdelrahman Sayed,HR,DB,Algorithms,AI,EDA,NaN,NaN,"HR, DB, Algorithms, AI, EDA, ,"
6,Ahmed Sakr,DB,Algorithms,HR,Signals,Compilers,AI,Testing,"DB, Algorithms, HR, Signals, Compilers, AI, Te..."
7,Osama Saad,Testing,AI,Algorithms,DSA,HR,DB,Signals,"Testing, AI, Algorithms, DSA, HR, DB, Signals"
8,Kareem Ramzy,DSA,Signals,HR,Testing,DB,Compilers,AI,"DSA, Signals, HR, Testing, DB, Compilers, AI"
9,Mina Shawket,HR,Algorithms,DB,Signals,Testing,DSA,NaN,"HR, Algorithms, DB, Signals, Testing, DSA,"


In [5]:
# To normalize the factors
min_factor = float(df_timetable["Factor"].min())
max_factor = float(df_timetable["Factor"].max())

In [6]:
def gamma(factor):
    return (factor - min_factor) / (max_factor - min_factor) * (MAX_GAP - MIN_GAP) + MIN_GAP

In [7]:
# Apply gamma to factor
df_timetable['recommended_gap'] = df_timetable["Factor"].apply(gamma)

In [8]:
# Join df_students and df_courses on the courses taken
def merge_timetable_and_students(df_students, df_timetable):
    merged_df = pd.melt(df_students,
                        id_vars=['Name'],
                        value_vars=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
                        var_name='Course',
                        value_name='C_Name')
    return pd.merge(merged_df,
                    df_timetable,
                    on='C_Name')

In [34]:
pd.melt(df_students,
                        id_vars=['Name'],
                        value_vars=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
                        var_name='Course',
                        value_name='C_Name')

,Name,Course,C_Name
0,Ziad Amerr,C1,Signals
1,Peter Ehab,C1,Signals
2,Mostafa Khalil,C1,Testing
3,Abdelrahman Salah,C1,Testing
4,Abdelrahman Soliman,C1,Signals
...,...,...,...
65,Abdelrahman Sayed,C7,NaN
66,Ahmed Sakr,C7,Testing
67,Osama Saad,C7,Signals
68,Kareem Ramzy,C7,AI


In [9]:
df = merge_timetable_and_students(df_students, df_timetable)

In [10]:
# Now let's check out one of the students
df.query("Name == 'Ziad Amerr'").head()

,Name,Course,C_Name,Code,Dept,ID,CH,Score,CW,Factor,Date,recommended_gap
0,Ziad Amerr,C1,Signals,ECE251,ECE,251,4,2.5,55,3.2,1,7.000000
13,Ziad Amerr,C5,Testing,CSE338,CSE,338,3,3.7,57,0.9,6,2.740741
25,Ziad Amerr,C7,HR,ASU111,ASU,111,3,3.3,53,2.1,2,4.962963
32,Ziad Amerr,C6,DB,CSE333,CSE,333,3,3.5,53,1.9,6,4.592593
39,Ziad Amerr,C2,AI,CSE472,CSE,472,3,3.5,57,1.1,8,3.111111


In [11]:
# Get the names of all students (should be shifted to their ID as the id is the primary key)
names = df["Name"].unique()

$$\Gamma(DepartmentFactor * CreditHours * ((1-\alpha) \cdot AverageCreditsLost + \alpha \cdot \Psi(AverageCourseworkMarksLost)))$$

$$BestGapFactor(C) = \Gamma(DepartmentFactor * CreditHours * ((1-\alpha) \cdot AverageCreditsLost + \alpha \cdot \Psi(AverageCourseworkMarksLost)))$$

In [12]:
timeslots_range = df["Date"].max() - df["Date"].min()
timeslots_range

7

In [13]:
df_timetable

,C_Name,Code,Dept,ID,CH,Score,CW,Factor,Date,recommended_gap
#,,,,,,,,,,
1,Signals,ECE251,ECE,251,4,2.5,55,3.2,1,7.000000
2,HR,ASU111,ASU,111,3,3.3,53,2.1,2,4.962963
3,EDA,CSE312,CSE,312,2,3.7,58,0.5,3,2.000000
4,DSA,CSE331,CSE,331,3,3.5,56,1.3,4,3.481481
5,Algorithms,CSE332,CSE,332,3,3.2,54,2.0,5,4.777778
6,DB,CSE333,CSE,333,3,3.5,53,1.9,6,4.592593
7,Testing,CSE338,CSE,338,3,3.7,57,0.9,6,2.740741
8,Compilers,CSE439,CSE,439,3,3.5,55,1.5,7,3.851852
9,AI,CSE472,CSE,472,3,3.5,57,1.1,8,3.111111


In [14]:
def get_courses_per_student_from_timetable(df_merged):
    # Let's divide the whole joined df to a dictionary of students
    courses_per_student = dict()

    # Filling our students' dictionary
    for name in names:
        curr_df = df_merged.query(f"Name == '{name}'")

        # Only capture the name of the course, its date and its recommended gap
        courses_per_student[name] = curr_df[["C_Name", "Date", "recommended_gap"]]
    return courses_per_student

In [15]:
courses_per_student = get_courses_per_student_from_timetable(df)

In [16]:
courses_per_student["Ziad Amerr"]

,C_Name,Date,recommended_gap
0,Signals,1,7.000000
13,Testing,6,2.740741
25,HR,2,4.962963
32,DB,6,4.592593
39,AI,8,3.111111
54,Algorithms,5,4.777778
57,Compilers,7,3.851852


In [17]:
def has_clash(courses):
    return True in set(courses["Date"].duplicated())

$$\Lambda(x, y) = (GapFactor(x) - (T(x) - T(y))^2$$

In [18]:
def compute_loss(curr_course, prev_course):
    # If we are testing against the first subject,
    # find the difference between it and the first
    # possible timeslot
    if prev_course is None:
        return curr_course["Date"] ** 2
    
    # Otherwise compute the loss normally
    return (curr_course["recommended_gap"] - (curr_course["Date"] - prev_course["Date"])) ** 2

$$J(S)=\sum_{i=0}^{S-1}\sum_{j=0}^{C(S_i)-1} \Lambda(C(S_i)_j, C(S_i)_{j-1})$$

In [19]:
def compute_cost(courses_per_student):
    """Computes the cost for the given students df"""
    cost = 0
    
    # Loop through each student
    for name, courses_of_that_student in courses_per_student.items():
        # Sort the courses taken by that student by date
        courses = courses_per_student[name].sort_values(by="Date")
        
        # Loop through each subject taken by that student
        loss = 0
        for j in range(0, len(courses)):
            curr_course = courses.iloc[j]
            if j==0:
                prev_course = None
            else:
                prev_course = courses.iloc[j-1]
            loss += compute_loss(curr_course, prev_course)
        cost += loss
    return cost

In [20]:
compute_cost(courses_per_student)

494.8010973936899

In [21]:
df.query("Name == 'Ziad Amerr'")

,Name,Course,C_Name,Code,Dept,ID,CH,Score,CW,Factor,Date,recommended_gap
0,Ziad Amerr,C1,Signals,ECE251,ECE,251,4,2.5,55,3.2,1,7.000000
13,Ziad Amerr,C5,Testing,CSE338,CSE,338,3,3.7,57,0.9,6,2.740741
25,Ziad Amerr,C7,HR,ASU111,ASU,111,3,3.3,53,2.1,2,4.962963
32,Ziad Amerr,C6,DB,CSE333,CSE,333,3,3.5,53,1.9,6,4.592593
39,Ziad Amerr,C2,AI,CSE472,CSE,472,3,3.5,57,1.1,8,3.111111
54,Ziad Amerr,C4,Algorithms,CSE332,CSE,332,3,3.2,54,2.0,5,4.777778
57,Ziad Amerr,C3,Compilers,CSE439,CSE,439,3,3.5,55,1.5,7,3.851852


In [22]:
df_timetable

,C_Name,Code,Dept,ID,CH,Score,CW,Factor,Date,recommended_gap
#,,,,,,,,,,
1,Signals,ECE251,ECE,251,4,2.5,55,3.2,1,7.000000
2,HR,ASU111,ASU,111,3,3.3,53,2.1,2,4.962963
3,EDA,CSE312,CSE,312,2,3.7,58,0.5,3,2.000000
4,DSA,CSE331,CSE,331,3,3.5,56,1.3,4,3.481481
5,Algorithms,CSE332,CSE,332,3,3.2,54,2.0,5,4.777778
6,DB,CSE333,CSE,333,3,3.5,53,1.9,6,4.592593
7,Testing,CSE338,CSE,338,3,3.7,57,0.9,6,2.740741
8,Compilers,CSE439,CSE,439,3,3.5,55,1.5,7,3.851852
9,AI,CSE472,CSE,472,3,3.5,57,1.1,8,3.111111


In [23]:
courses = df_timetable["C_Name"].unique()
courses

array(['Signals', 'HR', 'EDA', 'DSA', 'Algorithms', 'DB', 'Testing',
       'Compilers', 'AI'], dtype=object)

In [24]:
def generate_random_timetable(df_timetable):
    randomized = df_timetable.copy()
    randomized["Date"] = pd.Series(np.random.randint(0, MAX_TIMESLOT_DATE, size=N_COURSES, dtype='int16'))
    return randomized

In [25]:
def compute_cost_for_timetable(timetable):
    df = merge_timetable_and_students(df_students, timetable)
    courses_per_student = get_courses_per_student_from_timetable(df)
    
    has_clash_flag = False
    for name, student_timetable in courses_per_student.items():
        if has_clash(student_timetable):
            has_clash_flag = True
            break
    
    if has_clash_flag:
        return np.inf
    else:
        return compute_cost(courses_per_student)
    return cost

In [26]:
tt = generate_random_timetable(df_timetable)
tt

,C_Name,Code,Dept,ID,CH,Score,CW,Factor,Date,recommended_gap
#,,,,,,,,,,
1,Signals,ECE251,ECE,251,4,2.5,55,3.2,3,7.000000
2,HR,ASU111,ASU,111,3,3.3,53,2.1,4,4.962963
3,EDA,CSE312,CSE,312,2,3.7,58,0.5,5,2.000000
4,DSA,CSE331,CSE,331,3,3.5,56,1.3,2,3.481481
5,Algorithms,CSE332,CSE,332,3,3.2,54,2.0,6,4.777778
6,DB,CSE333,CSE,333,3,3.5,53,1.9,0,4.592593
7,Testing,CSE338,CSE,338,3,3.7,57,0.9,2,2.740741
8,Compilers,CSE439,CSE,439,3,3.5,55,1.5,3,3.851852
9,AI,CSE472,CSE,472,3,3.5,57,1.1,6,3.111111


In [27]:
compute_cost_for_timetable(tt)

inf

In [28]:
courses_per_student = get_courses_per_student_from_timetable(merge_timetable_and_students(df_students,
                                                                                          tt))

In [29]:
random_timetables = []

In [30]:
i = 0
while len(random_timetables) != 10:
    feasible_tt = generate_random_timetable(df_timetable)
    cost = compute_cost_for_timetable(feasible_tt)
    if not np.isinf(cost):
        random_timetables.append((i, feasible_tt))
        print(i, cost)
    i += 1

1872 760.1618655692729
3027 670.5473251028808
4957 648.278463648834
5183 720.8587105624142
6049 744.9972565157751
6599 658.9368998628257
12432 744.2606310013717
12988 463.753086419753
14931 737.1714677640604
18688 725.8655692729765


In [31]:
random_timetables = [tt[1] for tt in random_timetables]

In [32]:
costs = [compute_cost_for_timetable(tt) for tt in random_timetables]
costs

[760.1618655692729,
 670.5473251028808,
 648.278463648834,
 720.8587105624142,
 744.9972565157751,
 658.9368998628257,
 744.2606310013717,
 463.753086419753,
 737.1714677640604,
 725.8655692729765]

In [33]:
g_best = max(costs)
costs.find(g_best)

AttributeError: 'list' object has no attribute 'find'

In [ ]:
random_timetables[0][1]

In [ ]:
random_timetables[1][1].sort_values(by="Date")[["Date"]]

In [ ]:
random_timetables[4][1].sort_values(by="Date")